# Pre-processing

In [6]:
import pandas as pd

In [8]:
# tweets csv big dataset 2.1GB
file_path = 'bitcoin_tweets.csv'

# read tweet csv
# df_all = pd.read_csv(file_path)
# print(df_all.head())

columns = ['date', 'text']
# read two columns: date, text
df = pd.read_csv(file_path, engine="python", usecols=columns)
df.tail(10)

,date,text
4693081,2023-01-06 17:46:53,Jonas Cumberland #btc #彩票 Merle Noyes #世界杯直播 h...
4693082,2023-01-06 17:46:50,❤️ Join me at Bybit and earn exclusive rewards...
4693083,2023-01-06 17:46:50,❤️ Join me at Bybit and earn exclusive rewards...
4693084,2023-01-06 17:46:44,Lorraine Harvey #btc #彩票 Coral Bart #世界杯直播 htt...
4693085,2023-01-06 17:46:36,#DogelonMars is the future. #TSUKA is the nex...
4693086,2023-01-06 17:46:35,"Bitcoin squeeze is SUPER TIGHT, which way will..."
4693087,2023-01-06 17:46:29,Closed #BTC short at 16725. Missed my long pla...
4693088,2023-01-06 17:46:22,#Ethereum price update: \n\n#ETH $1263.59 USD\...
4693089,2023-01-06 17:46:20,1₿ = $16814.7 -0.07%🔻\n\nDetails:\nChange: 🔻-1...
4693090,2023-01-06 17:46:17,Earn crypto by playing fun games online.\nGet ...


Changing the data type of the Date column to datetime data type to support grouping based on *dates*

In [9]:
df = df[pd.to_datetime(df['date'], errors='coerce').notnull()]
df['date'] = pd.to_datetime(df['date'])

In [10]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /Users/yifan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
import re

def remove_hashtags(text):
    return re.sub(r'#\w+', '', text)

def remove_urls(text):
    return re.sub(r'http\S+|www\S+', '', text)

def remove_mentions(text):
    return re.sub(r'@\w+', '', text)

def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

def remove_urls(text):
    return re.sub(r"http\S+|www\S+|https\S+", '', text)


Call the functions to pre-process the Tweets column and store the pre-processed tweets to a seperate column
Processed Done:

* Remove Hashtags
* Remove URL's
* Remove Special Characters
* Remove the '\n' in the tweets

Using the functions created pre-process each dataset and store the pre-processed data to a seperate column labeled "preprocessed_tweets"

In [18]:
df['preprocessed_tweets'] = (df['text']
                             .apply(remove_hashtags)
                             .apply(remove_urls)
                             .apply(remove_mentions)
                             .apply(remove_special_characters)
                             .str.replace('\n', ' ', regex=False))

print(df.head(10))

                 date                                               text  \
0 2021-02-10 23:59:04  Blue Ridge Bank shares halted by NYSE after #b...   
1 2021-02-10 23:58:48  😎 Today, that's this #Thursday, we will do a "...   
2 2021-02-10 23:54:48  Guys evening, I have read this article about B...   
3 2021-02-10 23:54:33  $BTC A big chance in a billion! Price: \487264...   
4 2021-02-10 23:54:06  This network is secured by 9 508 nodes as of t...   
5 2021-02-10 23:53:30  💹 Trade #Crypto on #Binance \n\n📌 Enjoy #Cashb...   
6 2021-02-10 23:53:17  &lt;'fire' &amp; 'man'&gt;\n#Bitcoin #Crypto #...   
7 2021-02-10 23:52:42  🔄 Prices update in $EUR (1 hour):\n\n$BTC   - ...   
8 2021-02-10 23:52:25  #BTC #Bitcoin #Ethereum #ETH #Crypto #cryptotr...   
9 2021-02-10 23:52:08  .@Tesla’s #bitcoin investment is revolutionary...   

                                 preprocessed_tweets  
0  Blue Ridge Bank shares halted by NYSE after  A...  
1   Today thats this  we will do a  Take  with ou... 

# Sentiment Analysis

In [19]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()
df['sentiment_scores'] = df['preprocessed_tweets'].apply(lambda x: sid.polarity_scores(x)['compound'])

print(df.head(10))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/yifan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                 date                                               text  \
0 2021-02-10 23:59:04  Blue Ridge Bank shares halted by NYSE after #b...   
1 2021-02-10 23:58:48  😎 Today, that's this #Thursday, we will do a "...   
2 2021-02-10 23:54:48  Guys evening, I have read this article about B...   
3 2021-02-10 23:54:33  $BTC A big chance in a billion! Price: \487264...   
4 2021-02-10 23:54:06  This network is secured by 9 508 nodes as of t...   
5 2021-02-10 23:53:30  💹 Trade #Crypto on #Binance \n\n📌 Enjoy #Cashb...   
6 2021-02-10 23:53:17  &lt;'fire' &amp; 'man'&gt;\n#Bitcoin #Crypto #...   
7 2021-02-10 23:52:42  🔄 Prices update in $EUR (1 hour):\n\n$BTC   - ...   
8 2021-02-10 23:52:25  #BTC #Bitcoin #Ethereum #ETH #Crypto #cryptotr...   
9 2021-02-10 23:52:08  .@Tesla’s #bitcoin investment is revolutionary...   

                                 preprocessed_tweets  sentiment_scores  
0  Blue Ridge Bank shares halted by NYSE after  A...            0.2960  
1   Today thats t

In [20]:
# Using the functions created use them for each dataframe
# and then grouping them by their dates to get the sentiment score and label for each day
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
daily_sentiment = df.groupby('date')['sentiment_scores'].mean().reset_index()
print(daily_sentiment.head(10))

         date  sentiment_scores
0  2021-02-05          0.131285
1  2021-02-06          0.137975
2  2021-02-07          0.140346
3  2021-02-08          0.155133
4  2021-02-09          0.149979
5  2021-02-10          0.159520
6  2021-02-13          0.155729
7  2021-02-14          0.259754
8  2021-02-15          0.325989
9  2021-02-18          0.157838


In [36]:
# 定义开始日期和结束日期 (例如从 2021-01-01 到 2021-01-31)
# start_date = '2021-03-01'
# end_date = '2021-03-31'

# 筛选在时间范围内的数据
# df_filtered = daily_sentiment[(daily_sentiment['date'] >= start_date) & (daily_sentiment['date'] <= end_date)]
# df_filtered.to_csv('daily_sentiment.csv', index=False)


In [29]:
btc_data = pd.read_csv('btcusd_1-min_data.csv')
print(btc_data.head(10))

      Timestamp  Open  High   Low  Close  Volume
0  1.325412e+09  4.58  4.58  4.58   4.58     0.0
1  1.325412e+09  4.58  4.58  4.58   4.58     0.0
2  1.325412e+09  4.58  4.58  4.58   4.58     0.0
3  1.325412e+09  4.58  4.58  4.58   4.58     0.0
4  1.325412e+09  4.58  4.58  4.58   4.58     0.0
5  1.325412e+09  4.58  4.58  4.58   4.58     0.0
6  1.325412e+09  4.58  4.58  4.58   4.58     0.0
7  1.325412e+09  4.58  4.58  4.58   4.58     0.0
8  1.325413e+09  4.58  4.58  4.58   4.58     0.0
9  1.325413e+09  4.58  4.58  4.58   4.58     0.0


In [22]:
from datetime import datetime as dt

# defining function the that turn the timestamp to the date
def calculate_time(timestamp):
    """
    This function turns the timestamp to the date
    :param timestamp: given timestamp
    :return: date according to given timestamp
    """
    return dt.fromtimestamp(timestamp/1000)

In [34]:
# Turn "Timestamp" column to Date

# bitdate = []
# for i in btc_data["Timestamp"]:
#     bitdate.append(calculate_time(i))
# btc_data["bit_coin_date_time"] = bitdate

# 将时间戳列转换为datetime格式，无法解析的时间戳将变为NaT
btc_data['bit_coin_date_time'] = pd.to_datetime(btc_data['Timestamp'], unit='s', errors='coerce')
# btc_data.to_csv('bitcoin_date_time_price.csv', index=False)
# 删除包含非法时间戳（NaT）的行
btc_data = btc_data.dropna(subset=['bit_coin_date_time'])

# 定义开始日期和结束日期 (例如从 2021-01-01 到 2021-01-31)
# start_date = '2021-03-01'
# end_date = '2021-03-31'

# 筛选在时间范围内的数据
# df_filtered = btc_data[(btc_data['bit_coin_date_time'] >= start_date) & (btc_data['bit_coin_date_time'] <= end_date)]
# df_filtered.to_csv('bitcoin_valid_date_time_price.csv', index=False)

print(btc_data.head(10))

      Timestamp  Open  High   Low  Close  Volume  bit_coin_date_time
0  1.325412e+09  4.58  4.58  4.58   4.58     0.0 2012-01-01 10:01:00
1  1.325412e+09  4.58  4.58  4.58   4.58     0.0 2012-01-01 10:02:00
2  1.325412e+09  4.58  4.58  4.58   4.58     0.0 2012-01-01 10:03:00
3  1.325412e+09  4.58  4.58  4.58   4.58     0.0 2012-01-01 10:04:00
4  1.325412e+09  4.58  4.58  4.58   4.58     0.0 2012-01-01 10:05:00
5  1.325412e+09  4.58  4.58  4.58   4.58     0.0 2012-01-01 10:06:00
6  1.325412e+09  4.58  4.58  4.58   4.58     0.0 2012-01-01 10:07:00
7  1.325412e+09  4.58  4.58  4.58   4.58     0.0 2012-01-01 10:08:00
8  1.325413e+09  4.58  4.58  4.58   4.58     0.0 2012-01-01 10:09:00
9  1.325413e+09  4.58  4.58  4.58   4.58     0.0 2012-01-01 10:10:00


In [35]:
# group by date and get the last minute price of the day
df_daily_last = btc_data.groupby(btc_data['bit_coin_date_time'].dt.date).tail(1)

# save results to a new CSV file
df_daily_last.to_csv('bitcoin_daily_last_price.csv', index=False)

In [25]:
# process bitcoin data
btc_data = df_daily_last

btc_data['date'] = btc_data['bit_coin_date_time'].dt.strftime('%Y-%m-%d')
btc_data = btc_data[['date', 'Open', 'High', 'Low', 'Close', 'Volume']]
print(btc_data.head(10))

             date  Open  High   Low  Close  Volume
838    2012-01-01  4.84  4.84  4.84   4.84     0.0
2278   2012-01-02  5.00  5.00  5.00   5.00     0.0
3718   2012-01-03  5.29  5.29  5.29   5.29     0.0
5158   2012-01-04  5.57  5.57  5.57   5.57     0.0
6598   2012-01-05  6.42  6.42  6.42   6.42     0.0
8038   2012-01-06  6.40  6.40  6.40   6.40     0.0
9478   2012-01-07  6.80  6.80  6.80   6.80     0.0
10918  2012-01-08  6.90  6.90  6.90   6.90     0.0
12358  2012-01-09  6.48  6.48  6.48   6.48     0.0
13798  2012-01-10  7.10  7.10  7.10   7.10     0.0


In [26]:
# merge bitcoin data and sentiment data
merged_data = pd.merge(btc_data, daily_sentiment, on='date', how='inner')
print(merged_data.head(10))

         date      Open      High       Low     Close     Volume  \
0  2021-02-05  37494.08  37506.29  37372.59  37423.85  31.356678   
1  2021-02-06  40329.59  40362.46  40328.95  40341.59   0.369795   
2  2021-02-07  38371.71  38371.71  38319.25  38319.25   2.961660   
3  2021-02-08  42739.45  42739.45  42661.76  42705.78   1.481081   
4  2021-02-09  47062.03  47097.63  47015.62  47072.73  23.373384   
5  2021-02-10  45207.21  45210.00  45140.46  45144.15   1.940480   
6  2021-02-13  46829.35  46844.53  46787.05  46839.50   3.338138   
7  2021-02-14  48740.00  48764.35  48738.95  48764.35   0.444783   
8  2021-02-15  48479.10  48479.10  48344.27  48347.04   5.085603   
9  2021-02-18  51824.93  51877.15  51798.77  51800.00  25.976421   

   sentiment_scores  
0          0.131285  
1          0.137975  
2          0.140346  
3          0.155133  
4          0.149979  
5          0.159520  
6          0.155729  
7          0.259754  
8          0.325989  
9          0.157838  


In [27]:
merged_data.to_csv('bitcoin_price_sentiment.csv', index=False)